# Retrieval-Augmented Generation (RAG)

In [2]:
import chromadb
import dotenv
from pathlib import Path
from agents import Agent, Runner, function_tool, trace

dotenv.load_dotenv()

True

Create a static calorie table that we can use as a tool:

In [3]:
# We populated the RAG with the data from the data/calories.csv file in
# the rag_setup.ipynb notebook

chroma_client = chromadb.PersistentClient("../chroma")
nutrition_db = chroma_client.get_collection("nutrition_db")
nutrition_qna = chroma_client.get_collection("nutrition_qna")

In [6]:
results = nutrition_db.query(query_texts=["banana"], n_results=2)
for i, doc in enumerate(results["documents"][0]):
    print(sorted(results["metadatas"][0][i].items()))
    print(doc[:40] + "...")
    print("\n")

[('calories_per_100g', 89.0), ('food_category', 'fruits'), ('food_item', 'banana'), ('keywords', 'banana_fruits'), ('kj_per_100g', 374.0), ('serving_info', '100g')]
Food: Banana
        Category: Fruits
  ...


[('calories_per_100g', 50.0), ('food_category', '(fruit)juices'), ('food_item', 'banana juice'), ('keywords', 'banana_juice_(fruit)juices'), ('kj_per_100g', 210.0), ('serving_info', '100ml')]
Food: Banana Juice
        Category: (Fr...




In [7]:
results = nutrition_qna.query(query_texts=["banana"], n_results=2)
for i, doc in enumerate(results["documents"][0]):
    print(sorted(results["metadatas"][0][i].items()))
    print(doc + "...")
    print("\n")

[('is_pregnancy', False)]
Question: What is the recommended amount of bananas I should consume to count it as a single serving?
        Answer: One small-sized banana can be counted as a single serving.

        This Q&A pair provides information about nutrition and health topics....


[('is_pregnancy', False)]
Question: Which food is recommended for infants after they've been introduced to ripe bananas and sweet potatoes?
        Answer: Introduce porridge made from wheat flour or ground rice, starting with only one cereal. Once a week has passed, you may increase the frequency of this new food to two feedings per day.

        This Q&A pair provides information about nutrition and health topics....




In [30]:
@function_tool
def calorie_lookup_tool(query: str, max_results: int = 3) -> str:
    """
    Tool function for a RAG database to look up calorie information for specific food items, but not for meals.

    Args:
        query: The food item to look up.
        max_results: The maximum number of results to return.

    Returns:
        A string containing the nutrition information.
    """

    results = nutrition_db.query(query_texts=[query], n_results=max_results)

    if not results["documents"][0]:
        return f"No nutrition information found for: {query}"

    # Format results for the agent
    # FoodCategory,FoodItem,per100grams,Cals_per100grams,KJ_per100grams
    formatted_results = []
    for i, doc in enumerate(results["documents"][0]):
        metadata = results["metadatas"][0][i]
        food_item = metadata["food_item"].title()
        calories = metadata["calories_per_100g"]
        category = metadata["food_category"].title()

        formatted_results.append(
            f"{food_item} ({category}): {calories} calories per 100g"
        )

    return "Nutrition Information:\n" + "\n".join(formatted_results)

In [44]:
@function_tool
def question_lookup_tool(query: str, max_results: int = 3) -> str:
    """
    Tool function too ask a question about nutrition.

    Args:
        query: The question to ask
        max_results: The maximum number of results to return.

    Returns:
        A string containing the question and the answer related to the query.
    """

    results = nuntrition_qna_db.query(query_texts=[query], n_results=max_results)

    if not results["documents"][0]:
        return f"No information found for: {query}"

    # Format results for the agent
    formatted_results = []
    for i, doc in enumerate(results["documents"][0]):
        formatted_results.append(doc)

    return "Related answers to your question:\n" + "\n".join(formatted_results)

Let's test this out: 

_The following cell only works before you add the `@function_tool` annotation to `calorie_lookup_tool` function_

In [ ]:
#calorie_lookup_tool('bananas')

'Nutrition Information:\nBanana (Fruits): 89.0 calories per 100g\nBanana Juice ((Fruit)Juices): 50.0 calories per 100g\nBanana Nut Bread (Pastries,Breads&Rolls): 326.0 calories per 100g'

In [ ]:
#question_lookup_tool('bananas')

"Question Information:\nQuestion: Which food is recommended for infants after they've been introduced to ripe bananas and sweet potatoes?\n        Answer: Introduce porridge made from wheat flour or ground rice, starting with only one cereal. Once a week has passed, you may increase the frequency of this new food to two feedings per day.\n\n        This Q&A pair provides information about nutrition and health topics.\nQuestion: What is the recommended amount of bananas I should consume to count it as a single serving?\n        Answer: One small-sized banana can be counted as a single serving.\n\n        This Q&A pair provides information about nutrition and health topics.\nQuestion: Which fruits are high in potassium that I should consider eating?\n        Answer: Avocados, bananas, cantaloups, mangoes and papayas are all excellent options if you're looking for high-potassium fruits to eat.\n\n        This Q&A pair provides information about nutrition and health topics."

In [45]:
calorie_agent = Agent(
    name="Nutrition Assistant",
    instructions="""
    You are a helpful nutrition assistant giving out calorie information and nutrtion advice    .
    You give concise answers.
    
    If you need to look up calorie information, use the calorie_lookup_tool.
    If are asked a question about nutrition, always use the nutrtition_qna_tool first to see if there is an answer in the knowledge base.
    """,
    tools=[calorie_lookup_tool, question_lookup_tool],
)

In [46]:
with trace("Nutrition Assistant with Nutrition and Calorie RAG"):
    result = await Runner.run(
        calorie_agent,
        "What are the best meal choices for pregnant women and how many calories do they have?",
    )
    print(result.final_output)

Here are general guidance and some example meal ideas for pregnant women. Calorie needs vary by trimester and individual, so it’s best to consult your healthcare provider for personalized targets.

Key nutrition focus
- Protein: aim for lean meats, fish (low-mercury), eggs, dairy, beans, and nuts.
- Folate/iron: leafy greens, fortified cereals, beans, meat, lentils; vitamin C to help iron absorption.
- Calcium and vitamin D: dairy or fortified alternatives; salmon with bones, leafy greens.
- DHA/EPA: fatty fish (low-mercury) like salmon, sardines; consider a supplement if advised.
- Fiber and fluids: fruits, vegetables, whole grains; keep hydrated.
- Avoid: raw/undercooked seafood and eggs, unpasteurized dairy, certain soft cheeses, high-mercury fish, and unwashed produce.

Typical daily calorie guidance (adjust with your clinician)
- First trimester: no mandatory extra calories beyond a healthy, balanced diet.
- Second trimester: about +340–360 calories/day.
- Third trimester: about +